<a href="https://colab.research.google.com/github/pattycorporativo/CursoPO/blob/main/Curso_PO_AVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pesquisa Operacional

##Instalando o módulo python do gurobi

In [2]:
pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 90.3 MB/s eta 0:00:00


## Executando um exemplo

In [3]:
#!/usr/bin/env python3.11

# Copyright 2025, Gurobi Optimization, LLC

# This example formulates and solves the following simple MIP model:
#  maximize
#        x +   y + 2 z
#  subject to
#        x + 2 y + 3 z <= 4
#        x +   y       >= 1
#        x, y, z binary

import gurobipy as gp
from gurobipy import GRB

try:
    # Create a new model
    m = gp.Model("mip1")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"Obj: {m.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00

# Exercício Aula 1

In [ ]:
#!/usr/bin/env python3.11

# Copyright 2025, Gurobi Optimization, LLC

# Exemplo da aula para localização de bases


import gurobipy as gp
from gurobipy import GRB

try:
    # Create a new model
    m = gp.Model("mip1")

    matriz_a={
    [0,   0,   0,   0,   0,   1,   1,   0,   0,   1],
    [1,   0,   0,   0,   0,   0,   1,   0,   0,   1],
    [1,   0,   0,   1,   1,   1,   1,   0,   0,   0],
    [1,   1,   1,   0,   0,   0,   0,   1,   0,   1],
    [1,   0,   0,   0,   1,   1,   0,   0,   1,   1],
    [1,   0,   1,   0,   0,   1,   1,   1,   1,   1],
    [1,   1,   0,   0,   0,   0,   0,   0,   1,   0],
    [1,   1,   1,   1,   0,   1,   0,   1,   1,   1],
    [0,   1,   1,   0,   1,   0,   1,   1,   1,   1],
    [0,   1,   1,   1,   0,   0,   0,   1,   0,   1],
    [1,   0,   1,   0,   1,   0,   0,   0,   0,   1],
    [1,   1,   1,   1,   1,   0,   0,   1,   0,   0],
    [1,   0,   1,   1,   1,   0,   0,   1,   1,   1],
    [0,   1,   0,   1,   1,   1,   1,   1,   1,   1],
    [0,   1,   0,   1,   0,   0,   0,   0,   0,   1],
    [1,   1,   0,   1,   1,   0,   1,   1,   0,   1],
    [0,   1,   0,   0,   0,   0,   1,   0,   1,   0],
    [1,   1,   1,   0,   0,   1,   1,   1,   0,   1],
    [0,   0,   0,   1,   0,   0,   0,   0,   0,   1],
    [0,   1,   1,   0,   1,   1,   0,   0,   1,   1]
    }


except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")